In [1]:
import os

In [2]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file)
        
            logger.info(f"Downloaded {filename} from {self.config.source_URL}")
            logger.info(f"Size of downloaded file: {get_size(self.config.local_data_file)}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to {unzip_path}")

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-22 16:42:29,077: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-22 16:42:29,078: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-22 16:42:29,080: INFO: common: created directory at: artifacts]
[2025-10-22 16:42:29,081: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-22 16:42:29,083: INFO: 1583535059: File already exists at artifacts\data_ingestion\SamsumData.zip]
[2025-10-22 16:42:29,085: INFO: 1583535059: Extracted zip file to artifacts\data_ingestion]
